<a href="https://colab.research.google.com/github/MissMercyKN/AI-Tools/blob/main/Deep_Learning_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --upgrade pandas


In [2]:
pip install chardet

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import chardet
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt


Device Configuratiom

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Load and preprocess MNIST dataset

In [5]:
transform = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

 Define the CNN architecture

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # (B, 32, 28, 28)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # (B, 32, 14, 14)
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # (B, 64, 14, 14)
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                          # (B, 64, 7, 7)
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    # The forward method should be indented at the same level as __init__
    def forward(self, x):
        # The return statement should be indented relative to the forward definition
        return self.net(x)

model = CNN().to(device)

Define loss and optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Train the model

In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total, correct = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Calculate accuracy after the inner loop (processing all batches in an epoch)
    model.eval() # Set model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation
        for images, labels in train_loader: # Or use test_loader for test accuracy
             images, labels = images.to(device), labels.to(device)
             outputs = model(images)
             _, predicted = torch.max(outputs.data, 1)
             total += labels.size(0)
             correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Accuracy: {100 * correct / total:.2f}%")

Epoch [1/5], Train Accuracy: 98.36%
Epoch [2/5], Train Accuracy: 99.00%
Epoch [3/5], Train Accuracy: 99.21%


Evaluate on test set

In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")

 Visualize predictions on 5 test images

In [ ]:
import numpy as np

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

with torch.no_grad():
    output = model(example_data.to(device))
    pred = output.argmax(dim=1, keepdim=True)

 Plotting

In [ ]:
fig = plt.figure(figsize=(10, 4))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.tight_layout()
    plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
    plt.title(f"Pred: {pred[i].item()}")
    plt.xticks([])
    plt.yticks([])
plt.show()